In [66]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict



In [67]:


class State(TypedDict):
    runs : int
    balls : int
    fours: int
    sixes: int
    sr: float
    bpb: float
    boundary_percent: float
    summary : str

In [68]:
import stat


def calculate_sr(state:State):

    sr = state['runs']/state['balls']*100
    state['sr'] = sr
    return {"sr": sr}


def calculate_bpb(state:State):
    bpb = state['balls']/state['fours']+ state['sixes']
    # state['bpb'] = bpb
    return {"bpb":bpb}


def calculate_bp(state:State)-> State:
    boundary_percent = ((state['fours'] * 4)+(state['sixes']*6)/state['runs']) * 100
    state['boundary_percent'] = boundary_percent
    return {"boundary_percent": boundary_percent}

def summary(state:State)-> State:
    bp = ((state['fours'] * 4)+(state['sixes']*6)/state['runs']) * 100
    state['boundary_percent'] = bp
    summary  =  f"""
    Strike Rate - {state['sr']}\n
    Balls per Boundries - {state['boundary_percent']}\n

    """
    # state['summary'] = summary
    return {"summary":summary}

In [69]:
graph = StateGraph(State)

graph.add_node('calculate_sr',calculate_sr)
graph.add_node('calculate_bpb',calculate_bpb)
graph.add_node('calculate_bp',calculate_bp)
graph.add_node('summary',summary)

In [70]:
graph.add_edge(START, "calculate_sr")
graph.add_edge(START, "calculate_bpb")
graph.add_edge(START, "calculate_bp")

graph.add_edge("calculate_sr","summary")
graph.add_edge("calculate_bpb","summary")
graph.add_edge("calculate_bp","summary")

graph.add_edge("summary", END)

In [71]:
workflow = graph.compile()

initial_state = {
    "runs" : 100,
    "balls" : 50,
    "fours" : 6,
    "sixes" : 4
}

workflow.invoke(initial_state)


{'runs': 100,
 'balls': 50,
 'fours': 6,
 'sixes': 4,
 'sr': 200.0,
 'bpb': 12.333333333333334,
 'boundary_percent': 2424.0,
 'summary': '\n    Strike Rate - 200.0\n\n    Balls per Boundries - 2424.0\n\n\n    '}